In [1]:
!git clone https://github.com/fastai/course-v3/
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd /content/course-v3/nbs/dl2/
from exp.nb_05b import *

Cloning into 'course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.10 MiB | 38.33 MiB/s, done.
Resolving deltas: 100% (3251/3251), done.
/content/course-v3/nbs/dl2


In [2]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [3]:
import inspect; inspect.getsource(normalize)

'def normalize(x, m, s): return (x-m)/s\n'

In [4]:
x_train, y_train, x_valid, y_valid = get_data()
def normalize_to(x_train, x_valid):
    m, s = x_train.mean(), x_train.std()
    return (x_train-m)/s, (x_valid-m)/s
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1

In [5]:
# cnn
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x): return self.func(x)

def mnist_view(x): return x.view(-1, 1, 28, 28)

def flatten(x): return x.view(x.shape[0], -1)

def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )

# cuda
class CudaCallback(Callback):
    def begin_fit(self):
        self.model = self.model.cuda()
    def begin_batch(self):
        self.run.xb, self.run.yb = self.xb.cuda(), self.yb.cuda()    

In [6]:
device = torch.device('cuda', 0)
torch.cuda.set_device(device)

In [7]:
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback]

nfs = [8, 16, 32, 64, 64]
model = get_cnn_model(data)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)
run.fit(3, learn)

train: [1.83490671875, tensor(0.3439, device='cuda:0')]
valid: [0.51525888671875, tensor(0.8532, device='cuda:0')]
train: [0.4001935546875, tensor(0.8736, device='cuda:0')]
valid: [0.2177209716796875, tensor(0.9298, device='cuda:0')]
train: [0.21016666015625, tensor(0.9354, device='cuda:0')]
valid: [0.15620743408203125, tensor(0.9512, device='cuda:0')]


---

- A5

In [8]:
xb = next(iter(data.train_dl))[0].view(-1, 1, 28, 28)
model = nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        # Lambda(flatten),
        # nn.Linear(32, data.c)
    )
model(xb).shape

torch.Size([512, 32, 1, 1])

In [9]:
flatten(model(xb)).shape

torch.Size([512, 32])

In [ ]:
print(inspect.getsource(flatten))

def flatten(x): return x.view(x.shape[0], -1)



In [ ]:
nfs

[8, 16, 32, 64, 64]

In [10]:
def conv2d(ni, nf, ks, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, stride, padding=ks//2, **kwargs), nn.ReLU()
        )
def get_cnn_layers(nfs, data, **kwargs):
    nfs = [1] + nfs
    return [conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3, **kwargs) for i in range(len(nfs)-1)] + [
              nn.MaxPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

# we need to unpack the list since list is not a module subclass
def get_cnn_model(nfs, data, **kwargs):
    return nn.Sequential(*get_cnn_layers(nfs, data, **kwargs))

In [ ]:
get_cnn_model(nfs, data)

A7

In [ ]:
class batchXTransform():
    def __init__(self, func): self.tfms = func
    def begin_batch(self):
        self.run.xb = self.tfms(self.xb)
def view_tfms(*size):
    def _inner(x): x.view(*((-1,)+size)) #for example, in case of mnist, get 1,28, 28 and render -1, 1, 28, 28
    return _inner


A8

In [13]:
optim

<module 'torch.optim' from '/usr/local/lib/python3.7/dist-packages/torch/optim/__init__.py'>

In [20]:
def get_runner(model, data, lr=0.6, cbs=None, opt_func=optim.SGD, loss_func=F.cross_entropy):
    opt = opt_func(model.parameters(), lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner(cb_funcs=listify(cbs))
model = get_cnn_model(nfs, data)
learn, run = get_runner(model, data, lr=0.4, cbs=cbfs)
run.fit(1, learn)

RuntimeError: ignored

In [ ]:
|